## Setup

In [56]:
import torch
import random
import pandas as pd
import numpy as np

from ax.core.data import Data
from ax.core.experiment import Experiment
from ax.core.metric import Metric
from ax.core.objective import MultiObjective, Objective
from ax.core.optimization_config import (
    MultiObjectiveOptimizationConfig,
    ObjectiveThreshold,
)

from ax.core.parameter import ParameterType, RangeParameter, ChoiceParameter
from ax.core.search_space import SearchSpace
from ax.metrics.noisy_function import NoisyFunctionMetric

# Factory methods for creating multi-objective optimization modesl.
from ax.modelbridge.factory import get_MOO_PAREGO

# Analysis utilities, including a method to evaluate hypervolumes
from ax.modelbridge.modelbridge_utils import observed_hypervolume
from ax.modelbridge.registry import Models
from ax.runners.synthetic import SyntheticRunner
from ax.service.utils.report_utils import exp_to_df
from botorch.test_functions.multi_objective import BraninCurrin


## My Dataset

In [57]:
csv_path = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/"+\
"bpp-mse-psnr-loss-flops-params_bmshj2018-factorized_10000-epochs_D-3-4_L-2e-2-1e-2-5e-3_N-32-64-96-128-160-192-224_M-32-64-96-128-160-192-224-256-288-320.csv"
data = pd.read_csv(csv_path).set_index("labels")

In [58]:
# MAX_HV = 

ref_point = data[["bpp_loss","mse_loss","params"]].max().values * 1.1

x1 = ChoiceParameter(name="D", values=[3,4], parameter_type=ParameterType.INT, is_ordered=True, sort_values=True)
x2 = ChoiceParameter(name="L", values=[5e-3, 1e-2, 2e-2], parameter_type=ParameterType.FLOAT, is_ordered=True, sort_values=True)
x3 = ChoiceParameter(name="N", values=[32, 64, 96, 128, 160, 192, 224], parameter_type=ParameterType.INT, is_ordered=True, sort_values=True)
x4 = ChoiceParameter(name="M", values=[32, 64, 96, 128, 160, 192, 224, 256, 288, 320], parameter_type=ParameterType.INT, is_ordered=True, 
                     sort_values=True)

search_space = SearchSpace(parameters=[x1, x2, x3, x4])

def params_to_label(D,L,N,M):
    D = str(int(D))
    L = "5e-3" if L == 5e-3 else ("1e-2" if L == 1e-2 else "2e-2")
    N = str(int(N))
    M = str(int(M))
    return f"D{D}L{L}N{N}M{M}"

class MetricA(NoisyFunctionMetric):
    def f(self, x: np.ndarray) -> float:
        return float(data.loc[params_to_label(*x),"bpp_loss"])

class MetricB(NoisyFunctionMetric):
    def f(self, x: np.ndarray) -> float:
        return float(data.loc[params_to_label(*x),"mse_loss"])

class MetricC(NoisyFunctionMetric):
    def f(self, x: np.ndarray) -> float:
        return float(data.loc[params_to_label(*x),"params"]) 


metric_a = MetricA("a", ["x1", "x2", "x3", "x4"], noise_sd=0.0, lower_is_better=True)
metric_b = MetricB("b", ["x1", "x2", "x3", "x4"], noise_sd=0.0, lower_is_better=True)
metric_c = MetricB("c", ["x1", "x2", "x3", "x4"], noise_sd=0.0, lower_is_better=True)

mo = MultiObjective(
    objectives=[Objective(metric=metric_a), Objective(metric=metric_b), Objective(metric=metric_c)],
)

objective_thresholds = [
    ObjectiveThreshold(metric=metric, bound=val, relative=False)
    for metric, val in zip(mo.metrics, ref_point)
]

optimization_config = MultiObjectiveOptimizationConfig(
    objective=mo,
    objective_thresholds=objective_thresholds,
)

## Define experiment creation utilities

These construct our experiment, then initialize with Sobol points before we fit a Gaussian Process model to those initial points.

In [3]:
# Reasonable defaults for number of quasi-random initialization points and for subsequent model-generated trials.
N_INIT = 6
N_BATCH = 25

In [4]:
def build_experiment():
    experiment = Experiment(
        name="pareto_experiment",
        search_space=search_space,
        optimization_config=optimization_config,
        runner=SyntheticRunner(),
    )
    return experiment

In [5]:
## Initialize with Sobol samples
def initialize_experiment(experiment):
    sobol = Models.SOBOL(search_space=experiment.search_space, seed=1234)
    for _ in range(N_INIT):
        experiment.new_trial(sobol.gen(1)).run()
    return experiment.fetch_data()

# Sobol
We use quasirandom points as a fast baseline for evaluating the quality of our multi-objective optimization algorithms.

In [6]:
sobol_experiment = build_experiment()
sobol_data = initialize_experiment(sobol_experiment)

sobol_model = Models.SOBOL(
    experiment=sobol_experiment,
    data=sobol_data,
    seed=1234
)

In [17]:

sobol_hv_list = []
for i in range(N_BATCH):
    
    generator_run = sobol_model.gen(1)
    trial = sobol_experiment.new_trial(generator_run=generator_run)
    trial.run()
    exp_df = exp_to_df(sobol_experiment)
    outcomes = np.array(exp_df[["a", "b", "c"]], dtype=np.double)
    # Fit a GP-based model in order to calculate hypervolume.
    # We will not use this model to generate new points.
    dummy_model = Models.BOTORCH_MODULAR(
        experiment=sobol_experiment,
        data=sobol_experiment.fetch_data(),
    )
    try:
        hv = observed_hypervolume(modelbridge=dummy_model)
    except:
        hv = 0
        print("Failed to compute hv")
    sobol_hv_list.append(hv)
    print(f"Iteration: {i}, HV: {hv}")

sobol_outcomes = np.array(exp_to_df(sobol_experiment)[["a", "b", "c"]], dtype=np.double)

## qNEHVI
Noisy Expected Hypervolume Improvement. This is our current recommended algorithm for multi-objective optimization.

In [8]:
ehvi_experiment = build_experiment()
ehvi_data = initialize_experiment(ehvi_experiment)

In [16]:

ehvi_hv_list = []
ehvi_model = None
for i in range(N_BATCH):
    torch.manual_seed(1234)
    ehvi_model = Models.BOTORCH_MODULAR(
        experiment=ehvi_experiment,
        data=ehvi_data,
    )
    
    generator_run = ehvi_model.gen(1)
    trial = ehvi_experiment.new_trial(generator_run=generator_run)
    trial.run()
    ehvi_data = Data.from_multiple_data([ehvi_data, trial.fetch_data()])

    exp_df = exp_to_df(ehvi_experiment)
    outcomes = np.array(exp_df[["a", "b", "c"]], dtype=np.double)
    try:
        hv = observed_hypervolume(modelbridge=ehvi_model)
    except:
        hv = 0
        print("Failed to compute hv")
    ehvi_hv_list.append(hv)
    print(f"Iteration: {i}, HV: {hv}")

ehvi_outcomes = np.array(exp_to_df(ehvi_experiment)[["a", "b", "c"]], dtype=np.double)

## qNParEGO
This is a good alternative algorithm for multi-objective optimization when qNEHVI runs too slowly.

In [10]:
parego_experiment = build_experiment()
parego_data = initialize_experiment(parego_experiment)

In [15]:

parego_hv_list = []
parego_model = None
for i in range(N_BATCH):

    torch.manual_seed(1234)
    random.seed(1234)
    np.random.seed(1234)
    
    parego_model = get_MOO_PAREGO(
        experiment=parego_experiment,
        data=parego_data,
    )
    
    generator_run = parego_model.gen(1)
    trial = parego_experiment.new_trial(generator_run=generator_run)
    trial.run()
    parego_data = Data.from_multiple_data([parego_data, trial.fetch_data()])

    exp_df = exp_to_df(parego_experiment)
    outcomes = np.array(exp_df[["a", "b", "c"]], dtype=np.double)
    try:
        hv = observed_hypervolume(modelbridge=parego_model)
    except:
        hv = 0
        print("Failed to compute hv")
    parego_hv_list.append(hv)
    print(f"Iteration: {i}, HV: {hv}")

parego_outcomes = np.array(exp_to_df(parego_experiment)[["a", "b", "c"]], dtype=np.double)

# Hypervolume statistics
The hypervolume of the space dominated by points that dominate the reference point.

#### Plot the results
The plot below shows a common metric of multi-objective optimization performance when the true Pareto frontier is known:  the log difference between the hypervolume of the true Pareto front and the hypervolume of the approximate Pareto front identified by each algorithm. The log hypervolume difference is plotted at each step of the optimization for each of the algorithms.

The plot show that $q$NEHVI vastly outperforms $q$NParEGO which outperforms the Sobol baseline.

In [12]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.cm import ScalarMappable

%matplotlib inline

In [14]:
iters = np.arange(1, N_BATCH + 1)
log_hv_difference_sobol = np.log10(MAX_HV - np.asarray(sobol_hv_list))[
    : N_BATCH + 1
]
log_hv_difference_parego = np.log10(MAX_HV - np.asarray(parego_hv_list))[
    : N_BATCH + 1
]
log_hv_difference_ehvi = np.log10(MAX_HV - np.asarray(ehvi_hv_list))[
    : N_BATCH + 1
]

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.plot(iters, log_hv_difference_sobol, label="Sobol", linewidth=1.5)
ax.plot(iters, log_hv_difference_parego, label="qNParEGO", linewidth=1.5)
ax.plot(iters, log_hv_difference_ehvi, label="qNEHVI", linewidth=1.5)
ax.set(
    xlabel="number of observations (beyond initial points)",
    ylabel="Log Hypervolume Difference",
)
ax.legend(loc="lower right")